# NetworkChuck AI Chatbot - RAG Development & Testing
## Interactive development and testing of the RAG system

### 1. Setup and Imports

In [9]:
# === SETUP AND IMPORTS ===
import os
import sys
from pathlib import Path
from typing import List, Tuple
import logging

# Add project root to path
project_root = Path.cwd()
if 'notebooks' in str(project_root):
    project_root = project_root.parent
sys.path.append(str(project_root / 'src'))

# Third-party imports
from dotenv import load_dotenv
import gradio as gr
import openai

# LangChain imports
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone as LangchainPinecone
from langchain.schema import SystemMessage

# Load environment
load_dotenv()
print("✅ Setup complete!")

✅ Setup complete!


In [10]:
# === RAG RETRIEVER ===
class RAGRetriever:
    def __init__(self, index_name: str = "networkchuck-ai-chatbot"):
        self.index_name = index_name
        self.setup_components()
        
    def setup_components(self):
        # Initialize embeddings
        self.embeddings = OpenAIEmbeddings(
            model="text-embedding-3-small",
            openai_api_key=os.getenv('OPENAI_API_KEY')
        )
        
        # Connect to vectorstore
        self.vectorstore = LangchainPinecone.from_existing_index(
            index_name=self.index_name,
            embedding=self.embeddings
        )
        print("✅ RAG Retriever ready!")
    
    def retrieve_context(self, query: str, personality: str, top_k: int = 5):
        metadata_filter = {"personality": personality}
        docs = self.vectorstore.similarity_search_with_score(
            query=query, k=top_k, filter=metadata_filter
        )
        return [(doc, score) for doc, score in docs]
    
    def format_context(self, doc_score_pairs: List[Tuple], max_length: int = 3000):
        if not doc_score_pairs:
            return "No relevant context found."
        
        context_parts = []
        current_length = 0
        
        for doc, score in doc_score_pairs:
            video_title = doc.metadata.get('video_title', 'Unknown Video')
            timestamp = doc.metadata.get('start_time', 0)
            entry = f"[From: {video_title} at {timestamp}s] (Score: {score:.3f})\n{doc.page_content}\n\n"
            
            if current_length + len(entry) > max_length:
                break
            context_parts.append(entry)
            current_length += len(entry)
        
        return "".join(context_parts).strip()

In [11]:
# === PERSONALITY PROMPTS ===
class PersonalityPromptManager:
    def __init__(self):
        self.personalities = {
            "networkchuck": {
                "system_prompt": """You are NetworkChuck, an enthusiastic cybersecurity and networking expert who loves to teach technology in an engaging, hands-on way.

PERSONALITY TRAITS:
- Energetic and passionate about technology
- Uses casual, friendly language with occasional excitement
- Loves practical, hands-on demonstrations  
- Often mentions coffee and encourages learning
- Explains complex topics in simple terms
- Focuses on real-world applications

EXPERTISE: Networking, cybersecurity, Linux, cloud computing, Python, DevOps

RESPONSE STYLE:
- Start with enthusiasm
- Break down concepts step-by-step
- Include practical examples
- Use analogies to make concepts relatable
- Encourage hands-on practice
- End with motivation to keep learning"""
            },
            "bloomy": {
                "system_prompt": """You are Bloomy, a professional financial analyst and Excel expert with deep knowledge of Bloomberg Terminal and advanced financial modeling.

PERSONALITY TRAITS:
- Professional and analytical approach
- Precise and detail-oriented
- Focuses on practical financial applications
- Values efficiency and accuracy
- Explains complex financial concepts clearly
- Emphasizes best practices and industry standards

EXPERTISE: Bloomberg Terminal, Excel, financial modeling, VBA, Power Query, risk management

RESPONSE STYLE:
- Professional but approachable tone
- Structured, logical explanations
- Focus on practical financial applications
- Include specific function names and shortcuts
- Emphasize accuracy and best practices"""
            }
        }
        print("✅ Personality prompts loaded!")
    
    def build_prompt(self, personality: str, user_query: str, context: str):
        config = self.personalities.get(personality.lower())
        if not config:
            raise ValueError(f"Unknown personality: {personality}")
        
        return f"""{config['system_prompt']}

RELEVANT CONTEXT FROM YOUR VIDEOS:
{context}

USER QUESTION: {user_query}

Please respond as {personality.title()}, using the context from your videos while maintaining your authentic personality and teaching style."""

In [12]:
# === RAG ENGINE ===
class RAGEngine:
    def __init__(self):
        self.retriever = RAGRetriever()
        self.prompt_manager = PersonalityPromptManager()
        self.client = openai.OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
        print("✅ RAG Engine ready!")
    
    def generate_response(self, user_query: str, personality: str = "networkchuck"):
        try:
            # Retrieve context
            doc_score_pairs = self.retriever.retrieve_context(user_query, personality)
            context = self.retriever.format_context(doc_score_pairs)
            
            # Build prompt
            prompt = self.prompt_manager.build_prompt(personality, user_query, context)
            
            # Generate response
            response = self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "system", "content": prompt}],
                temperature=0 # 0.7
            )
            
            return {
                "response": response.choices[0].message.content,
                "context": context,
                "sources": len(doc_score_pairs),
                "personality": personality
            }
            
        except Exception as e:
            return {
                "response": f"Sorry, I encountered an error: {e}",
                "context": "",
                "sources": 0,
                "personality": personality
            }

# Initialize RAG Engine
rag_engine = RAGEngine()

✅ RAG Retriever ready!
✅ Personality prompts loaded!
✅ RAG Engine ready!


In [13]:
# === TEST FUNCTION ===
def test_query(query: str, personality: str = "networkchuck"):
    print(f"\n{'='*60}")
    print(f"🤖 Testing {personality.upper()} personality")
    print(f"❓ Query: {query}")
    print(f"{'='*60}")
    
    result = rag_engine.generate_response(query, personality)
    
    print(f"\n📊 RESULTS:")
    print(f"Sources found: {result['sources']}")
    print(f"Context length: {len(result['context'])} characters")
    print(f"\n🤖 RESPONSE:")
    print(result['response'])
    
    return result

print("✅ Test function ready!")

✅ Test function ready!


In [14]:
# === GRADIO INTERFACE ===
def create_gradio_interface():
    def chat_interface(message, personality, history):
        result = rag_engine.generate_response(message, personality.lower())
        history.append((message, result['response']))
        return history, ""
    
    with gr.Blocks(title="NetworkChuck AI Chatbot") as interface:
        gr.Markdown("# 🤖 NetworkChuck AI Chatbot")
        gr.Markdown("Choose a personality and start chatting!")
        
        with gr.Row():
            with gr.Column():
                personality = gr.Radio(
                    choices=["NetworkChuck", "Bloomy"],
                    value="NetworkChuck",
                    label="Choose Personality"
                )
                
                chatbot = gr.Chatbot(label="Chat History")
                msg = gr.Textbox(
                    label="Your Message",
                    placeholder="Ask me about networking, cybersecurity, Excel, or finance!",
                    lines=2
                )
                
                with gr.Row():
                    submit_btn = gr.Button("Send", variant="primary")
                    clear_btn = gr.Button("Clear Chat")
        
        # Event handlers
        submit_btn.click(chat_interface, [msg, personality, chatbot], [chatbot, msg])
        msg.submit(chat_interface, [msg, personality, chatbot], [chatbot, msg])
        clear_btn.click(lambda: ([], ""), outputs=[chatbot, msg])
    
    return interface

# Create interface
gradio_app = create_gradio_interface()
print("✅ Gradio interface created!")

✅ Gradio interface created!


C:\Users\jeand\AppData\Local\Temp\ipykernel_34712\1697031208.py:20: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Chat History")


## 🚀 Ready to Use!

### Test Individual Queries:
```python
test_query("How do I set up Docker containers?", "networkchuck")
test_query("How do I use VLOOKUP in Excel?", "bloomy")
```

### Launch Web Interface:
```python
gradio_app.launch()
```

In [17]:
# === LAUNCH ===
# Test it works
test_query("How to create a table in Excel", "bloomy")


🤖 Testing BLOOMY personality
❓ Query: How to create a table in Excel

📊 RESULTS:
Sources found: 5
Context length: 677 characters

🤖 RESPONSE:
To create a table in Excel, you can follow these steps:

1. Select the data range that you want to include in your table.
2. Go to the "Insert" tab on the Excel ribbon.
3. Click on the "Table" option. This will prompt a dialog box where Excel will automatically select the data range for your table.
4. Ensure that the "My table has headers" box is checked if your data includes headers.
5. Click "OK" to create the table.

By creating a table in Excel, you can easily manage and analyze your data. Tables offer features like structured formatting, automatic filtering, and dynamic ranges that can enhance your data analysis capabilities.

If you encounter any issues or have further questions about creating tables in Excel, feel free to ask for more guidance.


{'response': 'To create a table in Excel, you can follow these steps:\n\n1. Select the data range that you want to include in your table.\n2. Go to the "Insert" tab on the Excel ribbon.\n3. Click on the "Table" option. This will prompt a dialog box where Excel will automatically select the data range for your table.\n4. Ensure that the "My table has headers" box is checked if your data includes headers.\n5. Click "OK" to create the table.\n\nBy creating a table in Excel, you can easily manage and analyze your data. Tables offer features like structured formatting, automatic filtering, and dynamic ranges that can enhance your data analysis capabilities.\n\nIf you encounter any issues or have further questions about creating tables in Excel, feel free to ask for more guidance.',
 'context': '[From: How to use the Bloomberg Data Set (BDS) function in Excel (2 minutes) at 10.12s] (Score: 0.575)\nin Excel.\n\n[From: How to use the Bloomberg Query Language (BQL) Builder for panel data in Exc

In [16]:
# Launch web interface
gradio_app.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
